In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [ ]:
# Create a scatter plot for the trajectory
def ThreeDPlot(motion):
    fig = go.Figure(data=[go.Scatter3d(
        x=motion['x_global'],
        y=motion['y_global'],
        z=motion['z_global'],
        mode='lines',
        line=dict(color='blue', width=2),
    )]) 

    fig.update_layout(
        title='3D Trajectory Plot with Grid',
        scene=dict(
            xaxis=dict(title='X Position', gridcolor='lightgray', showgrid=True),
            yaxis=dict(title='Y Position', gridcolor='lightgray', showgrid=True),
            zaxis=dict(title='Z Position', gridcolor='lightgray', showgrid=True),
            aspectmode='cube'  # Optional: Keep the aspect ratio equal
        ),
        margin=dict(l=0, r=0, b=0, t=40)  # Adjust margins for better visibility
    )   

    # Show the interactive plot
    fig.show()

# check rotations
- rotation change is correct

In [ ]:
rot_y = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/rotation_checks/rotate_around_y-2024-12-29_15-09-51/Gyroscope.csv')
rot_z = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/rotation_checks/rotate_around_z-2024-12-29_15-08-55/Gyroscope.csv')
rot_x = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/rotation_checks/rotate_around_x-2024-12-29_15-07-15/Gyroscope.csv')

def calcAngles(df, title=''):
    df['time_diff_s'] = df['time'].diff().fillna(0) / 1e9
    df['roll'] = (df['x']*df['time_diff_s']).cumsum()
    df['pitch'] = (df['y']*df['time_diff_s']).cumsum()
    df['yaw'] = (df['z']*df['time_diff_s']).cumsum()
    df.drop(['x','y','z'],axis=1,inplace=True)

    if title != '':
        plt.figure()
        df[['pitch','yaw','roll']].plot()
        plt.legend()
        plt.title(title)

calcAngles(rot_x, 'around_x')
calcAngles(rot_y, 'around_y')
calcAngles(rot_z, 'around_z')

# only 2d motion for now (success)

In [ ]:
# rotation around y (=pitch)
# acceleration in x and z
# coosy: x = forwards, y = upward, z = to right
folder = 'Room_back_home-2024-12-29_19-50-26' #2024-11-09_15-06-37
gyro = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/'+folder+'/Gyroscope.csv')
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['roll'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['pitch'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['yaw'] = (gyro['z']*gyro['time_diff_s']).cumsum()
gyro.drop(['x','y','z'],axis=1,inplace=True)

acc = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/'+folder+'/Accelerometer.csv')

motion = pd.merge(gyro, acc, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)
motion

motion2 = motion.copy()
motion2.drop(['roll','yaw','ay'],axis=1,inplace=True)
motion2['vx_ego'] = (motion2['ax']*motion2['time_diff_s']).cumsum() # vx = vx0 + dt*ax
motion2['vz_ego'] = (motion2['az']*motion2['time_diff_s']).cumsum() # vx = vx0 + dt*ax

#motion2['x_ego'] = (motion2['ax']*motion2['time_diff_s']*motion2['time_diff_s']*0.5).cumsum()  #  x = x0 + 1/2*ax*dt^2
#motion2['z_ego'] = (motion2['az']*motion2['time_diff_s']*motion2['time_diff_s']*0.5).cumsum()  #  x = x0 + 1/2*ax*dt^2

motion2['dx_ego'] = (motion2['vx_ego']*motion2['time_diff_s'])#.cumsum()  #  x = x0 + 1/2*ax*dt^2
motion2['dz_ego'] = (motion2['vz_ego']*motion2['time_diff_s'])#.cumsum()  #  x = x0 + 1/2*ax*dt^2

motion2['x_global'] = (np.cos(motion2['pitch'])*motion2['dx_ego'] - np.sin(motion2['pitch'])*motion2['dz_ego']).cumsum()
motion2['z_global'] = (np.sin(motion2['pitch'])*motion2['dx_ego'] + np.cos(motion2['pitch'])*motion2['dz_ego']).cumsum()

motion2[['vx_ego','vz_ego']].plot()
plt.legend()
plt.figure()
motion2[['dx_ego','dz_ego']].plot()
plt.legend()
motion2[['pitch']].plot()
plt.legend()
motion2.plot(x='x_global',y='z_global')
plt.legend()


# 3d motion

In [ ]:
def euler_to_rotation_matrix(yaw, pitch, roll):
    # Convert degrees to radians if necessary
    #yaw = np.radians(yaw)
    #pitch = np.radians(pitch)
    #roll = np.radians(roll)

    # Rotation matrices around each axis
    R_yaw = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                      [np.sin(yaw), np.cos(yaw), 0],
                      [0, 0, 1]])

    R_pitch = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                        [0, 1, 0],
                        [-np.sin(pitch), 0, np.cos(pitch)]])

    R_roll = np.array([[1, 0, 0],
                       [0, np.cos(roll), -np.sin(roll)],
                       [0, np.sin(roll), np.cos(roll)]])

    # Combined rotation matrix (R = R_yaw * R_pitch * R_roll)
    R = R_yaw @ R_pitch @ R_roll
    return R

def transform_acceleration_to_global(dx,dy,dz, yaw, pitch, roll):
    R = euler_to_rotation_matrix(yaw, pitch, roll)
    #print(R, np.array((ax,ay,az)).reshape(3,1))
    gl = R @ np.array((dx,dy,dz)).reshape(3,1)
    return gl.flatten()


In [ ]:
# rotation around y (=pitch)
# acceleration in x and z
# coosy: x = forwards, y = upward, z = to right

folder = 'Room_back_home-2024-12-29_19-50-26' #2024-11-09_15-06-37
gyro = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/'+folder+'/Gyroscope.csv')
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['roll'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['pitch'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['yaw'] = (gyro['z']*gyro['time_diff_s']).cumsum()
gyro.drop(['x','y','z'],axis=1,inplace=True)

acc = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/'+folder+'/Accelerometer.csv')

motion = pd.merge(gyro, acc, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)
motion

motion['vx_ego'] = (motion['ax']*motion['time_diff_s']).cumsum() # vx = vx0 + dt*ax
motion['vy_ego'] = (motion['ay']*motion['time_diff_s']).cumsum()
motion['vz_ego'] = (motion['az']*motion['time_diff_s']).cumsum() # vx = vx0 + dt*ax

motion['dx_ego'] = (motion['vx_ego']*motion['time_diff_s'])
motion['dy_ego'] = (motion['vy_ego']*motion['time_diff_s'])
motion['dz_ego'] = (motion['vz_ego']*motion['time_diff_s'])

motion[['x_global', 'y_global', 'z_global']] = motion.apply(
    lambda row: transform_acceleration_to_global(row['ax'], row['ay'], row['az'],
                                                  row['yaw'], row['pitch'], row['roll']),
    axis=1,
    result_type='expand'  # This allows us to expand the result into multiple columns
)

motion['x_global'] = motion['x_global'].cumsum()
motion['y_global'] = motion['y_global'].cumsum()
motion['z_global'] = motion['z_global'].cumsum()


#motion[['vx_ego','vz_ego','vy_ego']].plot()
#plt.legend()
#plt.figure()
#motion[['dx_ego','dz_ego','dy_ego']].plot()
#plt.legend()
motion[['pitch','yaw','roll']].plot()
plt.legend()
motion.plot(x='x_global',y='z_global')
plt.title('birdseyeview')
plt.legend()
ThreeDPlot(motion)

# noise reduction via fourier transform

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def denoiseFFT(signal, fs = 1000, cutoff_frequency = 60, plot=False):
    # Apply FFT
    fft_signal = np.fft.fft(signal)
    frequencies = np.fft.fftfreq(motion.shape[0], 1/fs) 

    # Create a mask for filtering (e.g., remove frequencies above 60 Hz)
    fft_signal[np.abs(frequencies) > cutoff_frequency] = 0  

    # Inverse FFT to get the filtered signal
    filtered_signal = np.fft.ifft(fft_signal)   

    if plot:
        plt.figure(figsize=(12, 6))
        plt.subplot(3, 1, 1)
        plt.title('Original Signal with Noise')
        plt.plot(motion['ax'])
        plt.subplot(3, 1, 2)
        plt.title('Filtered Signal')
        plt.plot(filtered_signal.real)  # Use .real since IFFT may produce complex numbers
        plt.subplot(3, 1, 3)
        plt.title('Frequencies')
        plt.plot(fft_signal)  # Use .real since IFFT may produce complex numbers
        plt.tight_layout()
        plt.show()
    
    return filtered_signal.real

denoiseFFT(motion['global_az'], 1000, 60, True)


In [ ]:
# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the trajectory
ax.plot(motion['x_global'], motion['y_global'], motion['z_global'], label='Trajectory', color='b')
ax.set_title('3D Trajectory Plot')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Position')
ax.legend()

# Show the plot
plt.show()